In [2]:
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# %pip install -r requirements.txt
# %cd -

In [12]:
# !mkdir yolo
# !mkdir yolo/train
# !mkdir yolo/test

# !mkdir ./train_given_dset
# !mkdir ./test_given_dset

# !mkdir ./train_given_dset/tiger
# !mkdir ./train_given_dset/leopard
# !mkdir ./test_given_dset/tiger
# !mkdir ./test_given_dset/leopard

# !mkdir ./train_given_dset/tiger/Label
# !mkdir ./train_given_dset/leopard/Label
# !mkdir ./test_given_dset/tiger/Label
# !mkdir ./test_given_dset/leopard/Label

mkdir: cannot create directory ‘yolo’: File exists
mkdir: cannot create directory ‘yolo/train’: File exists


In [11]:
# from glob import glob
# import random

# data_dir="./Train_3500/3 класса/"
# tiger_imgs=sorted(glob(data_dir+'Тигры/*jpg'))
# tiger_labels=sorted(glob(data_dir+'Тигры/labels/*'))
# leopard_imgs=sorted(glob(data_dir+'дальневосточный леопард/*jpg'))
# leopard_labels=sorted(glob(data_dir+'дальневосточный леопард/labels/*'))

# tiger_imgs_labels = list(zip(tiger_imgs, tiger_labels))
# leopard_imgs_labels = list(zip(leopard_imgs, leopard_labels))

# random.shuffle(tiger_imgs_labels)
# random.shuffle(leopard_imgs_labels)

# train_tiger_imgs = [case[0] for case in tiger_imgs_labels[:2500]]
# train_tiger_labels = [case[1] for case in tiger_imgs_labels[:2500]]
# test_tiger_imgs = [case[0] for case in tiger_imgs_labels[2500:]]
# test_tiger_labels = [case[1] for case in tiger_imgs_labels[2500:]]

# train_leopard_imgs = [case[0] for case in leopard_imgs_labels[:2500]]
# train_leopard_labels = [case[1] for case in leopard_imgs_labels[:2500]]
# test_leopard_imgs = [case[0] for case in leopard_imgs_labels[2500:]]
# test_leopard_labels = [case[1] for case in leopard_imgs_labels[2500:]]

In [28]:
# import shutil

# for img_path in train_tiger_imgs:
#     shutil.copyfile(img_path, './train_given_dset/tiger/' + img_path.split('/')[-1])
# for img_path in test_tiger_imgs:
#     shutil.copyfile(img_path, './test_given_dset/tiger/' + img_path.split('/')[-1])
# for img_path in train_leopard_imgs:
#     shutil.copyfile(img_path, './train_given_dset/leopard/' + img_path.split('/')[-1])
# for img_path in test_leopard_imgs:
#     shutil.copyfile(img_path, './test_given_dset/leopard/' + img_path.split('/')[-1])
    
# for img_path in train_tiger_labels:
#     shutil.copyfile(img_path, './train_given_dset/tiger/Label/' + img_path.split('/')[-1])
# for img_path in test_tiger_labels:
#     shutil.copyfile(img_path, './test_given_dset/tiger/Label/' + img_path.split('/')[-1])
# for img_path in train_leopard_labels:
#     shutil.copyfile(img_path, './train_given_dset/leopard/Label/' + img_path.split('/')[-1])
# for img_path in test_leopard_labels:
#     shutil.copyfile(img_path, './test_given_dset/leopard/Label/' + img_path.split('/')[-1])

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from tqdm import tqdm
import os
import cv2
import matplotlib.pyplot as plt

data_dir="."
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

all_train_subdir=glob.glob(train_dir+"/*")
all_test_subdir=glob.glob(test_dir+"/*")

train_dir = os.path.join(data_dir, "train_given_dset")
test_dir = os.path.join(data_dir, "test_given_dset")

all_train_given_subdir=glob.glob(train_dir+"/*")
all_test_given_subdir=glob.glob(test_dir+"/*")

In [2]:
train_classes=[os.path.basename(pp) for pp in all_train_subdir] + [os.path.basename(pp) for pp in all_train_given_subdir]
test_classes=[os.path.basename(pp) for pp in all_test_subdir] + [os.path.basename(pp) for pp in all_test_given_subdir]

print("There is %d classes in train dataset, and %d classes in test dataset"%(len(train_classes), len(test_classes)))

There is 82 classes in train dataset, and 82 classes in test dataset


In [3]:
# yolo_train_dir = "yolo/train"
# yolo_test_dir = "yolo/test"

# given dataset train/test
# yolo_train_dir = "yolo/train_given"
# yolo_test_dir = "yolo/test_given"

# mixed dataset train/test
yolo_train_dir = "yolo/train_mixed"
yolo_test_dir = "yolo/test_mixed"

In [4]:
for dd in [yolo_train_dir, yolo_test_dir]:
    for ss in ["images", "labels"]:
        print(os.path.join(dd, ss))
        os.makedirs(os.path.join(dd, ss), exist_ok=True)

yolo/train_mixed/images
yolo/train_mixed/labels
yolo/test_mixed/images
yolo/test_mixed/labels


In [40]:
def process_dataset(subdirs, dst_dir, class_names, size=(640,640), link=False):
    for subdir_id in tqdm(range(len(subdirs))):
        subdir = subdirs[subdir_id]
        prefix=os.path.basename(subdir)
        for image_file in glob.glob(os.path.join(subdir, "*.jpg")):
            image_file_basename=os.path.basename(image_file)
            label_file = os.path.join(subdir, "Label", image_file_basename).replace(".jpg", ".txt")
            dst_image_file = os.path.join(dst_dir, "images/%s_%s"%(prefix,image_file_basename))
            dst_label_file = os.path.join(dst_dir, "labels/%s_%s"%(prefix,image_file_basename.replace(".jpg", ".txt")))
            if os.path.exists(dst_label_file):
                continue
            if not os.path.exists(label_file):
                continue
                        
            image = cv2.imread(image_file)                
            height, width = image.shape[0:2]
            with open(label_file) as fobj:
                with open(dst_label_file, "w") as wobj:
                    while True:
                        item = fobj.readline()
                        if item is None or len(item)==0:
                            break
                        class_name = prefix
                        item=item[len(class_name):]
                        item = item.split()
                        xmin = float(item[0])
                        ymin = float(item[1])
                        xmax = float(item[2])
                        ymax = float(item[3])

                        cx   = (xmin + xmax)/2.0/width
                        cy   = (ymin + ymax)/2.0/height
                        bw   = (xmax - xmin)/width
                        bh   = (ymax - ymin)/height
                        class_id = class_names.index(class_name)
                        output_line = "%d %f %f %f %f\n"%(class_id, cx, cy, bw, bh)
                        wobj.write(output_line)

            if link==True:
                os.symlink(image_file, dst_image_file)
            else:
                image = cv2.resize(image, size)
                cv2.imwrite(dst_image_file, image)
                
classes = ['bear', 'deer', 'fox', 'rabbit', 'lion', 'leopard', 'cheetah', 'tiger']
# process_dataset(all_train_subdir, yolo_train_dir, train_classes, size=(640,640), link=False) 

xueqin_train_subdir = [pp for pp in all_train_subdir if os.path.basename(pp).lower() in classes] + all_train_given_subdir
xueqin_classes=[os.path.basename(pp) for pp in xueqin_train_subdir]

process_dataset(xueqin_train_subdir, yolo_train_dir, xueqin_classes, size=(640,640), link=False)

100%|██████████| 2/2 [02:56<00:00, 88.00s/it]


In [41]:
# xueqin_test_subdir = [pp for pp in all_test_subdir if os.path.basename(pp).lower() in classes]
xueqin_test_subdir = all_train_subdir
process_dataset(xueqin_test_subdir, yolo_test_dir, xueqin_classes, size=(640,640), link=False)

100%|██████████| 2/2 [02:56<00:00, 88.12s/it]


In [42]:
yaml_file="yolov5/data/animal_given.yaml"
train_images_dir = os.path.join("..", yolo_train_dir, "images")
val_images_dir = os.path.join("..", yolo_test_dir, "images")

names_str=""
for item in xueqin_classes:
    names_str=names_str + ", \'%s\'"%item
names_str= "names: ["+names_str[1:]+"]"

with open(yaml_file, "w") as wobj:
    wobj.write("train: %s\n"%train_images_dir)
    wobj.write("val: %s\n"%val_images_dir)
    wobj.write("nc: %d\n"%len(xueqin_classes))
    wobj.write(names_str+"\n")

In [43]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sjeffry (use `wandb login --relogin` to force relogin)


True

In [44]:
%cd yolov5

/home/samara_hack_solution/yolov5


In [ ]:
!python train.py --data data/animal_given.yaml --batch-size 64 --epochs 60 --img-size 640 --project runs/train --name animals_given_xl --weights yolov5x.pt --device 0,1

wandb: Currently logged in as: sjeffry (use `wandb login --relogin` to force relogin)
train: weights=yolov5x.pt, cfg=, data=data/animal_given.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=60, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0,1, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=animals_given_xl, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-109-g7c6bae0 torch 1.10.0+cu102 CUDA:0 (Tesla V100S-PCIE-32GB, 32510MiB)
                                               CUDA:1 (Tesla V100S-PCIE-32GB, 32510MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.

In [56]:
!python detect.py --weights /home/samara_hack_solution/yolov5/runs/train/animals_given_medium/weights/best.pt --source /home/samara_hack_solution/test_given_dset/leopard/01180092_S02.jpg --name animals_given --project runs/detect

detect: weights=['/home/samara_hack_solution/yolov5/runs/train/animals_given_medium/weights/best.pt'], source=/home/samara_hack_solution/test_given_dset/leopard/01180092_S02.jpg, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=animals_given, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-109-g7c6bae0 torch 1.10.0+cu102 CUDA:0 (Tesla V100S-PCIE-32GB, 32510MiB)

Fusing layers... 
Model Summary: 290 layers, 20856975 parameters, 0 gradients, 48.0 GFLOPs
image 1/1 /home/samara_hack_solution/test_given_dset/leopard/01180092_S02.jpg: 480x640 1 leopard, Done. (0.008s)
Speed: 0.5ms pre-process, 8.4ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/animals_given3
